# RAG : Usando FAISS (Facebook AI Similarity Search = búsqueda de similitudes con IA de Facebook) como base de conocimiento

In [1]:
import os, getpass

Faiss es una biblioteca para la búsqueda eficiente de similitudes y agrupación de vectores. Contiene algoritmos que buscan en conjuntos de vectores de cualquier tamaño, hasta aquellos que posiblemente no quepan en la RAM. También contiene código de soporte para evaluación y ajuste de parámetros.

Faiss está escrito en C++ con contenedores completos para Python. Algunos de los algoritmos más útiles se implementan en la GPU. Está desarrollado por Facebook AI Research .

## Generando fragmentos de un archivo PDF

![Imagen](https://www.muycomputer.com/wp-content/uploads/2022/08/LOTR.jpg)

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("pdf/ElSeñordelosAnillos.pdf") #1753 páginas
data = loader.load()

#Genera varios fragmentos de 400 tokens
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 400, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(documents = data)

KeyboardInterrupt: 

In [ ]:
len(docs)

1628

In [ ]:
docs[0]

Document(page_content='El Señor de los Anillos es una novela, obra cumbre del escritor John\nRonald Reuel Tolkien, ambientada en un mundo fantástico llamado la\nTierra Media hacia el final de su Tercera Edad. El título hace\nreferencia a Sauron, el Señor Oscuro de Mordor, principal villano de\nla historia, creador del Anillo Único que utilizó para controlar el poder\nde los demás Anillos. El Señor de los Anillos constituye la\ncontinuación de un libro anterior de Tolkien, El Hobbit, que cuenta la\nhistoria de cómo el Anillo del Poder pasa a las manos de Bilbo\nBolsón, el tío de Frodo Bolsón. En un nivel más profundo, constituye\nla continuación de otro libro llamado El Silmarillion, que habla de la\ncreación de la Tierra Media y de todas las criaturas que en ella\nhabitan, así como del primer Señor Oscuro, maestro de Sauron, y de\nlas luchas por los Silmarils.\nEl Señor de los Anillos narra las aventuras de un grupo de seres:\n(elfos, hobbits, enanos, humanos), que forman la Comunidad 

## Cargando datos a FAISS

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2")
faiss_index = FAISS.from_documents(docs, embedding_function)

#from langchain.embeddings.openai import OpenAIEmbeddings
#embedding_function = OpenAIEmbeddings
#faiss_index = FAISS.from_documents(docs, embedding_function)

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
docs = faiss_index.similarity_search("¿a quién le pertenece el anillo?", k = 3)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

33: La Comunidad del Anillo
Primera parte de
El Señor de los Anillos
583: Las Dos Torres
Segunda parte de
El Señor de los Anillos
79: —Este es el Dueño de los Anillos, el Anillo Único que los gobierna.
Este es el Anillo Único que el Señor Oscuro perdió en tiempos remotos,
junto con parte de su pode r. Lo desea terriblemente, pero es necesario que no
lo consiga.
Frodo se sentó en silencio, inmóvil: el miedo parecía extender una man


In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingresa tu API Key de OpenAI : ")

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

template = """Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
{context}
Pregunta: {question}
Respuesta: 
"""

prompt = PromptTemplate(
    template = template, input_variables = ["context", "question"]
)

llm = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
    temperature = 0.0
)

qa = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = faiss_index.as_retriever(), #Por defecto recupera los 4 documentos más relevantes as_retriever(search_kwargs={'k': 3 })
    chain_type_kwargs = {"prompt": prompt, "verbose": True},
    verbose = True
)

In [11]:
qa.run("¿Puedes comparar y contrastar las personalidades de Frodo y Sam?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
que Frodo y Sam habían esperado. Sam lo alcanzó antes de que pudiera dar
dos pasos más. Frodo, qu e lo seguía, le aferró la pierna y lo hizo caer.
—Tu cuerda podr á sernos útil otra vez, Sam —di jo. Sam sacó la cuerda.
—¿Y a dónde iba usted por estas duras tierras frías, señor Gollum? —
gruñó—. Nos preguntamos, sí, nos preguntamos. En busca de algunos de tus
amigos orcos, apuesto. Repugnante criatura traicionera. Alrededor de tu
gaznate tendría que ir esta cuerda y con un nudo bi en apretado.
Gollum yacía inmóvil y no intentó ninguna otra jugarreta. No le contestó
a Sam, pero le echó una mirada fugaz y venenosa.
—Sólo nos hace falta algo con que sujeta

'Sí, puedo comparar y contrastar las personalidades de Frodo y Sam.\n\nFrodo es valiente y determinado, pero también muestra signos de debilidad y duda. A lo largo de la historia, se enfrenta a numerosos desafíos y tentaciones, pero sigue adelante con su misión de destruir el Anillo. Frodo es consciente de los peligros que enfrenta y se preocupa por la seguridad de sus amigos. A veces, se siente abrumado por la carga del Anillo y duda de su capacidad para completar la tarea.\n\nPor otro lado, Sam es leal y valiente. Aunque es un personaje secundario, desempeña un papel crucial en la historia. Sam está dispuesto a hacer cualquier cosa por Frodo y lo acompaña en su viaje a pesar de los peligros que enfrenta. A diferencia de Frodo, Sam no muestra dudas o debilidades significativas. Es un personaje más optimista y siempre está dispuesto a ayudar y apoyar a Frodo.\n\nEn resumen, Frodo es más complejo y muestra una gama más amplia de emociones y dudas, mientras que Sam es más leal y optimist

In [12]:
qa.run("¿Qué simboliza el anillo?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
Las Dos Torres
Segunda parte de
El Señor de los Anillos

—¡El Anillo! —exclamó Frodo—. ¿Me ha dejado el Anillo? Me
pregunto por  qué. Bueno, qu izá me sirva de algo.
—Sí y no —dijo Gandalf—. En tu lugar, yo no lo usaría. Pero guárdalo
en secreto ¡y en sitio seguro! Bien, me voy a la cama.
Como amo de Bolsón Cerrado, Frodo sintió que era su penoso deber
despedir a los huéspedes. Rumores sobre extraños acontecimientos se habían
diseminado por el campo. Frodo nada dijo, pero sin duda todo se aclararía
por la mañana . Alrededor de medianoche comenzaron a llegar los carruajes
de la gente importante y así fueron desapareciendo, uno a uno, cargados con
hobbit

'El anillo simboliza el poder y el control que busca el Señor Oscuro, así como la amenaza que representa para todos.'

## Preguntando a OpenAI

In [13]:
llm = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
    temperature = 0.0
)

llm.predict("¿Qué simboliza el anillo?")

'El anillo puede simbolizar diferentes cosas dependiendo del contexto y la cultura. Algunos de los significados más comunes asociados con los anillos son:\n\n- Compromiso y matrimonio: En muchas culturas, el anillo de compromiso y la alianza matrimonial simbolizan el compromiso y la unión entre dos personas.\n\n- Poder y autoridad: En algunas culturas, los anillos pueden ser símbolos de poder y autoridad, como los anillos de los reyes y reinas.\n\n- Amistad y lealtad: Los anillos también pueden ser símbolos de amistad y lealtad, como los anillos de la amistad que se intercambian entre amigos cercanos.\n\n- Herencia y legado: Los anillos familiares o de herencia pueden simbolizar la conexión con generaciones pasadas y el legado familiar.\n\n- Estatus social: En algunas culturas, los anillos pueden ser símbolos de estatus social, como los anillos de graduación o los anillos de pertenencia a una organización exclusiva.\n\nEstos son solo algunos ejemplos de los significados simbólicos que 